# 06: Human-in-the-Loop (Interactive)

When the AI is uncertain, actually ask a human for help!

In [8]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from config.llm_factory import LLMFactory
from pydantic import BaseModel
from typing import Optional

print("✅ Ready for interactive human review!")

✅ Ready for interactive human review!


## Simple Interactive BS Detector

In [9]:
class BSResult(BaseModel):
    verdict: str
    confidence: int
    reasoning: str
    human_reviewed: bool = False

def check_claim_with_human(claim: str, auto_mode: bool = False) -> BSResult:
    """Check claim and ask human when uncertain"""
    
    # Get AI verdict first
    llm = LLMFactory.create_llm()
    llm_with_structure = llm.with_structured_output(BSResult)
    
    prompt = f"""Analyze if this claim is BS or LEGITIMATE: {claim}
    
    Be skeptical. If you're not certain, use lower confidence.
    Confidence scale: 0-100 (below 60 means uncertain)"""
    
    ai_result = llm_with_structure.invoke(prompt)
    
    print(f"\n🤖 AI Analysis:")
    print(f"Verdict: {ai_result.verdict} ({ai_result.confidence}%)")
    print(f"Reasoning: {ai_result.reasoning}")
    
    # Check if human review needed
    if ai_result.confidence < 60:
        print(f"\n⚠️  Low confidence ({ai_result.confidence}%) - Human review needed!")
        
        if auto_mode:
            print("[Auto mode: Simulating human review]")
            return BSResult(
                verdict=ai_result.verdict,
                confidence=85,
                reasoning=f"Human reviewed: {ai_result.reasoning}",
                human_reviewed=True
            )
        else:
            # ACTUALLY ASK THE HUMAN
            print("\n" + "="*50)
            print("🧑 HUMAN REVIEW REQUIRED")
            print("="*50)
            print(f"\nClaim: '{claim}'")
            print(f"\nAI thinks: {ai_result.verdict} but only {ai_result.confidence}% confident")
            print(f"Reason: {ai_result.reasoning}")
            
            # Get human input
            print("\nWhat's your verdict?")
            human_verdict = input("Enter BS or LEGITIMATE: ").strip().upper()
            
            while human_verdict not in ["BS", "LEGITIMATE"]:
                human_verdict = input("Please enter BS or LEGITIMATE: ").strip().upper()
            
            human_confidence = int(input("Your confidence (0-100): "))
            human_reasoning = input("Brief explanation (optional): ") or "Human expert review"
            
            return BSResult(
                verdict=human_verdict,
                confidence=human_confidence,
                reasoning=f"Human: {human_reasoning}",
                human_reviewed=True
            )
    
    return ai_result

## Test with Auto Mode First

In [11]:
# Test in auto mode (simulates human review)
test_claims = [
    "The Boeing 747 has four engines",  # High confidence - no review needed
    "People think Boeing is not a trustworthy company anymore.",  # Should trigger review
]

print("🤖 Testing in AUTO MODE (simulated human review)\n")

for claim in test_claims:
    print(f"\n{'='*60}")
    print(f"Claim: '{claim}'")
    result = check_claim_with_human(claim, auto_mode=True)
    print(f"\nFinal verdict: {result.verdict} ({result.confidence}%)")
    print(f"Human reviewed: {result.human_reviewed}")

🤖 Testing in AUTO MODE (simulated human review)


Claim: 'The Boeing 747 has four engines'

🤖 AI Analysis:
Verdict: LEGITIMATE (95%)
Reasoning: The Boeing 747 is a well-known wide-body commercial aircraft that is designed and manufactured by Boeing. It is widely recognized for having four engines mounted under its wings. This fact is well-documented in aviation literature and by multiple reputable sources. Given the consistency of this information and its common knowledge in aviation, the claim that the Boeing 747 has four engines is legitimate.

Final verdict: LEGITIMATE (95%)
Human reviewed: True

Claim: 'People think Boeing is not a trustworthy company anymore.'

🤖 AI Analysis:
Verdict: LEGITIMATE (75%)
Reasoning: There have been well-publicized incidents involving Boeing, such as the 737 MAX crashes and subsequent investigations into their safety practices, which have led to public skepticism about the company's trustworthiness. While Boeing remains a major aerospace company, these

## Interactive Mode - You Provide Input!

In [12]:
# Now with REAL human input
print("👤 INTERACTIVE MODE - You will be asked for input!\n")

# Try a claim that should trigger human review
uncertain_claim = "Quantum computers can break all encryption instantly"

print(f"Testing claim: '{uncertain_claim}'")
result = check_claim_with_human(uncertain_claim, auto_mode=False)

print(f"\n✅ Final Result:")
print(f"Verdict: {result.verdict} ({result.confidence}%)")
print(f"Human reviewed: {result.human_reviewed}")
print(f"Reasoning: {result.reasoning}")

👤 INTERACTIVE MODE - You will be asked for input!

Testing claim: 'Quantum computers can break all encryption instantly'

🤖 AI Analysis:
Verdict: BS (85%)
Reasoning: Quantum computers have the potential to break certain encryption schemes, such as those based on integer factorization (e.g., RSA) or discrete logarithms, by using Shor's algorithm. However, current quantum computers are not yet powerful or error-corrected enough to break all encryption instantly. Moreover, symmetric encryption schemes like AES are generally more resilient, requiring a doubling of key length to maintain security against quantum attacks. Additionally, post-quantum cryptography efforts aim to develop quantum-resistant algorithms. Therefore, the claim that quantum computers can break all encryption instantly is an overstatement and misleading.

✅ Final Result:
Verdict: BS (85%)
Human reviewed: True
Reasoning: Quantum computers have the potential to break certain encryption schemes, such as those based on inte

## Interactive Loop

In [ ]:
print("🎮 Interactive BS Detector (type 'quit' to exit)\n")

while True:
    claim = input("\nEnter a claim to check: ")
    if claim.lower() == 'quit':
        break
    
    result = check_claim_with_human(claim, auto_mode=False)
    
    print(f"\n📊 Final Result:")
    print(f"Verdict: {result.verdict} ({result.confidence}%)")
    if result.human_reviewed:
        print("✋ Human reviewed")
    print(f"Reasoning: {result.reasoning}")

print("\nThanks for participating!")

## Production Implementation Ideas

1. **Web Interface**: Replace `input()` with web form
2. **Queue System**: Store uncertain claims for batch review
3. **Multiple Reviewers**: Get consensus from multiple humans
4. **Learn from Feedback**: Update model based on human corrections
5. **Audit Trail**: Log all human decisions for compliance